In [ ]:
import numpy as np
import Utilityfunctions as utils
import jax.numpy as jnp
import ssr_kronvec_jax as ssr
import ssr_likelihood_jax as jax_lik
import vanilla as mhn
import scipy.optimize as opt
import regularized_optimization as reg_opt

In [ ]:
n = 10
theta = utils.random_theta(n, 0.0)
n_dat = 50
dat = np.random.binomial(n=1, p=0.6, size=n_dat*(2*n+1))
dat = jnp.array(dat).reshape((n_dat, -1))
dat = dat.at[:,-1].set(1)
dat = jnp.column_stack((dat, jnp.ones(n_dat, dtype=int)))
lam1 = 1
lam2 = 1
indep = np.eye(n+1)
dat

In [ ]:
reg_opt.log_prob_coupled(theta.flatten(), dat, lam1, lam2, 1/n_dat)

In [ ]:
reg_opt.log_prob_single(theta.flatten(), dat, lam1, 1/n_dat)

In [ ]:
reg_opt.grad_single(indep.flatten(), dat, lam1, 1/n_dat)

In [ ]:
reg_opt.grad_coupled(indep.flatten(), dat, lam1, lam2, 1/n_dat)

In [ ]:
reg_opt.grad_coupled_2(indep.flatten(), dat, lam1, lam2, 1/n_dat)

In [ ]:
x = opt.minimize(reg_opt.log_prob_coupled, x0 = indep.flatten(), args = (dat, lam1, lam2, 0.07), 
                method = "L-BFGS-B", jac = reg_opt.grad_coupled_2, options={"maxiter":100, "disp":True})

In [ ]:
jnp.around(jnp.exp(jnp.reshape(x.x, (n+1, n+1))),2)